In [1]:
# import libraries
import numpy as np, pandas as pd
from scipy.optimize import minimize
import mpmath
from ortools.linear_solver import pywraplp

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head()

,srch_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score,prop_accesibility_score,prop_log_historical_price,price_usd,promotion_flag,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,booking_bool
0,1,4,3,1,2,0,5,140,0,0,4,0,1,1,0
1,1,3,5,1,2,0,5,211,0,0,4,0,1,1,0
2,1,4,4,1,3,0,5,150,0,0,4,0,1,1,0
3,1,4,3,1,3,0,5,144,0,0,4,0,1,1,0
4,1,4,4,1,2,0,5,191,0,0,4,0,1,1,0


In [4]:
mean_original = []
std_original = []
for i in range(1,9):
    mean = np.mean(df.iloc[:,i])
    mean_original.append(mean)
    std = np.std(df.iloc[:,i])
    std_original.append(std)

In [5]:
def get_Beta(df, mean, std):
    for i in range(1,9):
        df.iloc[:,i] = (df.iloc[:,i] - mean[i-1]) / std[i-1]
    search_id = df['srch_id'].unique().tolist()
    new_df = []
    for i in search_id:
        new_df.append(pd.DataFrame(df.loc[df['srch_id'] == i]).to_numpy(dtype=np.float32))
    def MLE(beta):
        log = 0
        for i in range(len(search_id)):
            linear_term = 0
            if 1 in new_df[i][:,14]:
                linear_term = new_df[i][:,1:9].dot(beta[1:]).dot(new_df[i][:,14]) + beta[0]
            log += linear_term
            total = 0
            for k in range(new_df[i].shape[0]):
                total +=  mpmath.exp(new_df[i][k][1:9].dot(beta[1:]) + beta[0])
            log -= float(mpmath.log(1 + total))
        return -log
    res = minimize(MLE, np.ones((9,)),  method='BFGS')
    return res

In [133]:
res_original = get_Beta(df,mean_original,std_original)

In [135]:
res_original

      fun: 20611.32596621373
 hess_inv: array([[ 2.50921995e-03, -1.99446847e-03, -2.76420143e-04,
        -2.13896877e-04, -1.58119281e-03,  4.40247951e-04,
        -2.99601098e-05,  6.09981189e-04,  1.93814867e-04],
       [-1.99446847e-03,  1.59105873e-03,  2.20026156e-04,
         1.64882623e-04,  1.25187108e-03, -3.53126349e-04,
         2.52519988e-05, -4.86355819e-04, -1.56296073e-04],
       [-2.76420143e-04,  2.20026156e-04,  3.13752311e-05,
         2.32078973e-05,  1.73196241e-04, -4.84238979e-05,
         3.39143093e-06, -6.57255868e-05, -2.27577366e-05],
       [-2.13896877e-04,  1.64882623e-04,  2.32078973e-05,
         2.33306138e-05,  1.39633428e-04, -3.47108048e-05,
         1.13316361e-06, -5.03066426e-05, -1.49335213e-05],
       [-1.58119281e-03,  1.25187108e-03,  1.73196241e-04,
         1.39633428e-04,  1.00458613e-03, -2.76952043e-04,
         1.67531261e-05, -3.85871110e-04, -1.16032865e-04],
       [ 4.40247951e-04, -3.53126349e-04, -4.84238979e-05,
        -3.

In [6]:
beta_original = [-1.74662743,  0.40812391,  0.10876074,  0.10138235,  0.02202491,
        0.04344406, -0.06686938, -1.33110656,  0.15977641]

In [175]:
def get_Assortment(df, mean, std, beta, theta):
    # df is data1,2,3,4
    def calculate_V():
        V = np.zeros((df.shape[0],len(theta)))
        
        for k in range(len(theta)):
            new_df = pd.DataFrame.copy(df)
            for i in range(8):
                new_df.iloc[:,i] = (new_df.iloc[:,i] - mean[k][i]) / std[k][i]
            V[:,k] = new_df.dot(beta[k][1:]) + beta[k][0]
        return np.exp(V)
    n = df.shape[0]
    p = df.iloc[:,6].to_numpy(dtype=np.float32)
    V = calculate_V()
    solver = pywraplp.Solver('final project',
                             pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    z = [0 for k in range(len(theta))]
    for k in range(len(theta)):
        z[k] = solver.NumVar(0, solver.infinity(), 'z' + str(k))
    x = [0 for i in range(n)]
    for j in range(n):
        x[j] = solver.IntVar(0, 1, 'x' + str(j))

    y = [[0 for k in range(len(theta))] for j in range(n)]
    for k in range(len(theta)):
        for j in range(n):
            y[j][k] = solver.NumVar(0, solver.infinity(), 'y' + str(j) + str(k))

    for k in range(len(theta)):
        sum_y = 0
        sum_x = 0
        for j in range(n):
            sum_y += V[j][k] * y[j][k]
            sum_x += V[j][k] * x[j] * p[j]
        solver.Add(z[k] + sum_y >= sum_x)
        solver.Add(z[k] + sum_y <= sum_x)

    M = 10000
    for j in range(n):
        for k in range(len(theta)):
            solver.Add(y[j][k] >= 0)
            solver.Add(y[j][k] <= M * x[j])

    for j in range(n):
        for k in range(len(theta)):
            solver.Add(- M * (1 - x[j]) + z[k] <= y[j][k])
            solver.Add(y[j][k] <= z[k])

    objective = solver.Objective()
    for i in range(2):
        objective.SetCoefficient(z[i], theta[i])
    objective.SetMaximization()
    status = solver.Solve()
    if status == solver.OPTIMAL:
        print('Problem solved in %f milliseconds' %solver.wall_time())
    elif status == solver.FEASIBLE:
        print('Solver claims feasibility but not optimality')
    else:
        print('Solver ran to completion but did not find an optimal solution')

    print('The objective value is ', objective.Value())
    assortment = []
    for i in range(n):
        assortment.append(x[i].solution_value())
    return assortment

In [11]:
data = pd.read_csv('data.csv')
df = data[['srch_id', 'srch_booking_window', 'srch_adults_count', 'srch_children_count',
       'srch_room_count', 'srch_saturday_night_bool', 'booking_bool']]
df.drop_duplicates(keep=False,inplace=True)
features = ['srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'booking_bool']

X = df[features].values
import sklearn.cluster
import sklearn.metrics
model_1 = sklearn.cluster.KMeans(n_clusters=3)
model_1.fit_predict(X)
results = model_1.labels_.tolist()
df1 = pd.DataFrame.copy(df)
df1['label'] = results

/Users/aaronyan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
model_2 = sklearn.cluster.KMeans(n_clusters=4)
model_2.fit_predict(X)
results_2 = model_2.labels_.tolist()
df2 = pd.DataFrame.copy(df)
df2['labels'] = results_2

In [18]:
df1.head()

,srch_id,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,booking_bool,label
9,1,0,4,0,1,1,1,0
43,75,30,3,1,1,0,1,1
77,101,74,4,1,2,1,1,1
91,218,0,1,1,2,0,1,0
100,236,15,3,0,1,0,1,0


In [19]:
df2.head()

,srch_id,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,booking_bool,labels
9,1,0,4,0,1,1,1,0
43,75,30,3,1,1,0,1,2
77,101,74,4,1,2,1,1,3
91,218,0,1,1,2,0,1,0
100,236,15,3,0,1,0,1,0


In [20]:
label_0 = []
label_1 = []
label_2 = []
for i in range(df1.shape[0]):
    if df1.iloc[i,-1] == 0.0:
        label_0.append(df1.iloc[i,0])
    elif df1.iloc[i,-1] == 1.0:
        label_1.append(df1.iloc[i,0])
    elif df1.iloc[i,-1] == 2.0:
        label_2.append(df1.iloc[i,0])

In [23]:
label0 = []
label1 = []
label2 = []
label3 = []
for i in range(df2.shape[0]):
    if df2.iloc[i,-1] == 0.0:
        label0.append(df2.iloc[i,0])
    elif df2.iloc[i,-1] == 1.0:
        label1.append(df2.iloc[i,0])
    elif df2.iloc[i,-1] == 2.0:
        label2.append(df2.iloc[i,0])
    elif df2.iloc[i,-1] == 3.0:
        label3.append(df2.iloc[i,0])

In [22]:
data_3cluster = pd.read_csv('data.csv')

# search_id = df1['srch_id'].unique().tolist()
# for i in search_id:
#     if data.iloc[]
data_3cluster['label'] = 0
for i in range (data_3cluster.shape[0]):
    if data_3cluster.iloc[i,0] in label_0:
        data_3cluster.iloc[i, -1] = 0
    elif data_3cluster.iloc[i,0] in label_1:
        data_3cluster.iloc[i, -1] = 1
    elif data_3cluster.iloc[i,0] in label_2:
        data_3cluster.iloc[i, -1] = 2


In [153]:
data_spec = pd.read_csv('spec_data.csv')

In [155]:
df_spec_one = data_spec[data_spec['labels'] == 0]
df_spec_two = data_spec[data_spec['labels'] == 1]
df_spec_three = data_spec[data_spec['labels'] == 2]
df_spec_four = data_spec[data_spec['labels'] == 3]

In [159]:
mean_spec_one = []
std_spec_one = []
for i in range(1,9):
    mean = np.mean(df_spec_one.iloc[:,i])
    mean_spec_one.append(mean)
    std = np.std(df_spec_one.iloc[:,i])
    std_spec_one.append(std)
mean_spec_two = []
std_spec_two = []
for i in range(1,9):
    mean = np.mean(df_spec_two.iloc[:,i])
    mean_spec_two.append(mean)
    std = np.std(df_spec_two.iloc[:,i])
    std_spec_two.append(std)
mean_spec_three = []
std_spec_three = []
for i in range(1,9):
    mean = np.mean(df_spec_three.iloc[:,i])
    mean_spec_three.append(mean)
    std = np.std(df_spec_three.iloc[:,i])
    std_spec_three.append(std)
mean_spec_four = []
std_spec_four = []
for i in range(1,9):
    mean = np.mean(df_spec_four.iloc[:,i])
    mean_spec_four.append(mean)
    std = np.std(df_spec_four.iloc[:,i])
    std_spec_four.append(std)


In [160]:
theta_spec_cluster = []
theta_spec_cluster.append(df_spec_one.shape[0] / data_spec.shape[0])
theta_spec_cluster.append(df_spec_two.shape[0] / data_spec.shape[0])
theta_spec_cluster.append(df_spec_three.shape[0] / data_spec.shape[0])
theta_spec_cluster.append(df_spec_four.shape[0] / data_spec.shape[0])

In [163]:
res_spec_1 = get_Beta(df_spec_one,mean_spec_one,std_spec_one)
res_spec_2 = get_Beta(df_spec_two,mean_spec_two,std_spec_two)
res_spec_3 = get_Beta(df_spec_three,mean_spec_three,std_spec_three)
res_spec_4 = get_Beta(df_spec_four,mean_spec_four,std_spec_four)

/Users/aaronyan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/aaronyan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/aaronyan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [164]:
res_spec_1

      fun: 2480.909782880881
 hess_inv: array([[ 8.67970611e+02, -9.59340057e-05, -9.58106959e-03,
        -3.14296218e-03,  7.66531622e-02, -2.25418338e-05,
         1.09550887e-01,  6.13322604e+02,  6.27235782e-04],
       [-9.71709406e-05,  1.65021835e-03, -6.89754678e-04,
        -2.93713539e-08, -5.03744500e-04,  1.86438553e-07,
        -1.14749551e-03, -5.57330923e-04, -2.03678355e-05],
       [-9.58077637e-03, -6.89755729e-04,  1.52774802e-03,
        -2.08231683e-05,  5.25163072e-05, -7.33160794e-10,
        -6.05148054e-04, -1.15600182e-02,  9.61175114e-06],
       [-3.14187975e-03, -2.79449586e-08, -2.08244228e-05,
         1.06315705e-04,  2.02853274e-05,  5.65530535e-08,
        -6.57820712e-05, -2.33493905e-03,  3.27672405e-06],
       [ 7.66551656e-02, -5.03718844e-04,  5.25085531e-05,
         2.02652461e-05,  8.95649863e-03, -1.59620594e-06,
        -8.22314238e-04,  3.72199535e-02, -3.34451409e-05],
       [-2.25394455e-05,  1.86435480e-07, -7.34461197e-10,
         5.

In [165]:
res_spec_2

      fun: 391.78289399894874
 hess_inv: array([[ 2.13484710e-01, -1.86260197e-01,  1.74566961e-01,
        -3.96586732e-02,  7.49386125e-02, -2.87512855e-03,
        -5.58015475e-01,  1.90758117e+00, -7.41375044e-02],
       [-1.86260197e-01,  1.70292033e-01, -1.55379544e-01,
         3.50919305e-02, -6.19746827e-02,  2.48586359e-03,
         4.82225305e-01, -1.66600192e+00,  6.61365003e-02],
       [ 1.74566961e-01, -1.55379544e-01,  1.45793819e-01,
        -3.33211709e-02,  5.90107205e-02, -2.35172769e-03,
        -4.60820502e-01,  1.57557074e+00, -6.22505676e-02],
       [-3.96586732e-02,  3.50919305e-02, -3.33211709e-02,
         8.86048097e-03, -1.34856674e-02,  5.57828932e-04,
         1.08901196e-01, -3.66746673e-01,  1.44732788e-02],
       [ 7.49386125e-02, -6.19746827e-02,  5.90107205e-02,
        -1.34856674e-02,  3.04797708e-02, -1.02873655e-03,
        -2.00560886e-01,  6.76747525e-01, -2.61519852e-02],
       [-2.87512855e-03,  2.48586359e-03, -2.35172769e-03,
         5

In [166]:
res_spec_3

      fun: 830.4197133013757
 hess_inv: array([[ 2.32918573e+00, -5.06436667e+00,  1.26842066e+00,
        -2.62736922e-03,  4.94601635e-02, -9.05469145e-05,
         2.02072822e+01,  1.00541810e+01,  9.19262119e-04],
       [-5.06436667e+00,  1.19186944e+01, -2.96588647e+00,
         1.43737749e-02, -5.65690651e-02,  4.89843165e-04,
        -4.73542613e+01, -2.36205616e+01, -3.99142359e-03],
       [ 1.26842066e+00, -2.96588647e+00,  7.42127450e-01,
        -3.59251625e-03,  1.38318632e-02, -1.01746510e-04,
         1.17786691e+01,  5.87550269e+00,  1.03259884e-03],
       [-2.62736922e-03,  1.43737749e-02, -3.59251625e-03,
         1.20282694e-03,  5.32939965e-04, -1.25861271e-05,
        -5.61910748e-02, -2.86284774e-02,  1.42617606e-04],
       [ 4.94601635e-02, -5.65690651e-02,  1.38318632e-02,
         5.32939965e-04,  1.19668364e-02,  4.65947185e-06,
         2.22973171e-01,  9.95197490e-02, -1.44101365e-04],
       [-9.05469145e-05,  4.89843165e-04, -1.01746510e-04,
        -1.

In [167]:
res_spec_4

      fun: 16819.90831930047
 hess_inv: array([[ 3.47783106e-06,  7.30148375e-08,  1.52425639e-07,
         6.11554944e-07, -9.72351117e-07, -3.25420425e-07,
        -5.49264292e-07,  7.87350923e-07, -1.09494252e-07],
       [ 7.30148375e-08,  1.49549006e-07, -3.65753282e-09,
         1.20651730e-07, -1.77101828e-07, -8.83220892e-08,
        -1.29053604e-07, -2.94997862e-07, -5.23587126e-09],
       [ 1.52425639e-07, -3.65753282e-09,  6.25141118e-07,
         9.18124126e-09,  7.38483358e-08, -5.43958193e-09,
        -2.40560831e-07, -6.31543522e-07, -2.51037119e-07],
       [ 6.11554944e-07,  1.20651730e-07,  9.18124126e-09,
         6.27347864e-07, -4.79107306e-07, -1.97601646e-07,
        -1.80312009e-07, -4.81831369e-07, -3.25795474e-08],
       [-9.72351117e-07, -1.77101828e-07,  7.38483358e-08,
        -4.79107306e-07,  9.12309448e-07,  1.86569420e-07,
         5.70408906e-08,  6.88057538e-07,  4.47265023e-09],
       [-3.25420425e-07, -8.83220892e-08, -5.43958193e-09,
        -1.

In [32]:
data_4cluster = pd.read_csv('kmeans_data.csv')

In [29]:
df_3_one = data_3cluster[data_3cluster['label'] == 0]
df_3_two = data_3cluster[data_3cluster['label'] == 1]
df_3_three = data_3cluster[data_3cluster['label'] == 2]

In [33]:
df_4_one = data_4cluster[data_4cluster['labels'] == 0]
df_4_two = data_4cluster[data_4cluster['labels'] == 1]
df_4_three = data_4cluster[data_4cluster['labels'] == 2]
df_4_four = data_4cluster[data_4cluster['labels'] == 3]

In [36]:
mean_3_one = []
std_3_one = []
for i in range(1,9):
    mean = np.mean(df_3_one.iloc[:,i])
    mean_3_one.append(mean)
    std = np.std(df_3_one.iloc[:,i])
    std_3_one.append(std)
mean_3_two = []
std_3_two = []
for i in range(1,9):
    mean = np.mean(df_3_two.iloc[:,i])
    mean_3_two.append(mean)
    std = np.std(df_3_two.iloc[:,i])
    std_3_two.append(std)
mean_3_three = []
std_3_three = []
for i in range(1,9):
    mean = np.mean(df_3_three.iloc[:,i])
    mean_3_three.append(mean)
    std = np.std(df_3_three.iloc[:,i])
    std_3_three.append(std)


In [37]:
mean_4_one = []
std_4_one = []
for i in range(1,9):
    mean = np.mean(df_4_one.iloc[:,i])
    mean_4_one.append(mean)
    std = np.std(df_4_one.iloc[:,i])
    std_4_one.append(std)
mean_4_two = []
std_4_two = []
for i in range(1,9):
    mean = np.mean(df_4_two.iloc[:,i])
    mean_4_two.append(mean)
    std = np.std(df_4_two.iloc[:,i])
    std_4_two.append(std)
mean_4_three = []
std_4_three = []
for i in range(1,9):
    mean = np.mean(df_4_three.iloc[:,i])
    mean_4_three.append(mean)
    std = np.std(df_4_three.iloc[:,i])
    std_4_three.append(std)
mean_4_four = []
std_4_four = []
for i in range(1,9):
    mean = np.mean(df_4_three.iloc[:,i])
    mean_4_four.append(mean)
    std = np.std(df_4_three.iloc[:,i])
    std_4_four.append(std)


In [117]:
res_one = get_Beta(df_3_one,mean_3_one,std_3_one)
res_two = get_Beta(df_3_two,mean_3_two,std_3_two)
res_three = get_Beta(df_3_three,mean_3_three,std_3_three)

/Users/aaronyan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/aaronyan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/aaronyan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [118]:
res_one

      fun: 16229.354198152256
 hess_inv: array([[ 2.02668753e-04,  2.86324006e-05, -7.12824220e-05,
        -5.92436894e-05, -6.12220825e-05, -3.02275505e-05,
        -9.46361659e-05,  1.33514447e-04,  8.01000211e-05],
       [ 2.86324006e-05,  1.66961007e-04, -7.87849689e-05,
        -8.17313015e-06,  4.01727279e-05, -2.71781084e-05,
        -1.03893385e-04,  1.42625283e-06, -2.31733114e-05],
       [-7.12824220e-05, -7.87849689e-05,  2.71886382e-04,
        -1.41743403e-05, -2.08183014e-04,  3.58955616e-05,
         2.49469755e-04, -3.48378840e-05, -6.38881845e-05],
       [-5.92436894e-05, -8.17313015e-06, -1.41743403e-05,
         2.54258762e-04, -6.22637876e-06,  4.24407920e-06,
         9.43505970e-05, -1.82953258e-04, -9.65790898e-06],
       [-6.12220825e-05,  4.01727279e-05, -2.08183014e-04,
        -6.22637876e-06,  2.90971473e-04, -2.78008507e-05,
        -2.02579909e-04, -1.89406806e-05, -7.62554695e-08],
       [-3.02275505e-05, -2.71781084e-05,  3.58955616e-05,
         4

In [119]:
res_two

      fun: 2621.703410283225
 hess_inv: array([[ 2.83578660e-03,  5.69998840e-04, -1.27996789e-03,
        -1.54894236e-04, -5.39562800e-04,  1.21208277e-04,
         1.09471286e-03, -1.40513116e-03,  2.01713617e-04],
       [ 5.69998840e-04,  7.93590884e-04, -8.24868698e-04,
         1.92750090e-04,  7.49015496e-05, -9.71954109e-05,
        -2.28183955e-04, -2.51611860e-04, -2.74411076e-04],
       [-1.27996789e-03, -8.24868698e-04,  1.94623315e-03,
         4.72610628e-05, -4.85738831e-04, -1.01660361e-04,
        -2.58420826e-05,  4.82318151e-04,  3.71459763e-04],
       [-1.54894236e-04,  1.92750090e-04,  4.72610628e-05,
         5.36976304e-04,  8.07284784e-05, -1.47599689e-04,
        -1.17390921e-03,  1.22657826e-04, -7.81053414e-05],
       [-5.39562800e-04,  7.49015496e-05, -4.85738831e-04,
         8.07284784e-05,  6.27039120e-04,  3.18590314e-05,
        -6.53174550e-04,  4.06111756e-04, -3.42939304e-04],
       [ 1.21208277e-04, -9.71954109e-05, -1.01660361e-04,
        -1.

In [120]:
res_three

      fun: 1105.7512765205931
 hess_inv: array([[ 2.47537127e-02,  3.69093869e-03, -1.07800709e-03,
         1.02641255e-03,  3.41269434e-03, -3.53152345e-03,
        -4.16136865e-03, -1.09692788e-02,  2.13962974e-03],
       [ 3.69093869e-03,  1.92715300e-03, -6.06078006e-04,
        -1.13410482e-03, -4.10486173e-05, -1.24491056e-03,
        -1.17052586e-03, -6.84869351e-04,  1.05452514e-03],
       [-1.07800709e-03, -6.06078006e-04,  2.05659447e-03,
        -5.18710058e-04,  9.93016541e-04, -3.10115202e-04,
        -4.67165861e-04, -3.49769241e-05,  5.59772340e-05],
       [ 1.02641255e-03, -1.13410482e-03, -5.18710058e-04,
         4.02014428e-03, -3.82790737e-04,  1.10455462e-03,
         1.67506610e-03, -1.91013278e-03, -1.98602174e-03],
       [ 3.41269434e-03, -4.10486173e-05,  9.93016541e-04,
        -3.82790737e-04,  2.09849247e-03, -8.33680369e-04,
        -1.12582860e-03, -2.12248593e-03,  9.19523879e-04],
       [-3.53152345e-03, -1.24491056e-03, -3.10115202e-04,
         1

In [45]:
res_1 = get_Beta(df_4_one,mean_4_one,std_4_one)
res_2 = get_Beta(df_4_two,mean_4_two,std_4_two)
res_3 = get_Beta(df_4_three,mean_4_three,std_4_three)
res_4 = get_Beta(df_4_four,mean_4_four,std_4_four)

/Users/aaronyan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/aaronyan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/aaronyan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [46]:
res_1

      fun: 13769.503055670977
 hess_inv: array([[ 3.55922994e-04,  1.53718615e-04, -1.83997204e-04,
         1.02878530e-04, -4.72598459e-05, -1.32023599e-04,
        -5.33858916e-05,  3.17387356e-04,  6.29731768e-05],
       [ 1.53718615e-04,  4.38181716e-04, -1.72925957e-04,
         1.22144257e-04,  9.03010873e-05,  3.51360778e-05,
         2.63906559e-04, -3.95875700e-05,  2.38964707e-05],
       [-1.83997204e-04, -1.72925957e-04,  3.13124429e-04,
        -4.82958200e-05, -1.44001827e-04,  1.00355855e-04,
        -2.99959854e-05, -2.22110453e-04, -1.27494707e-04],
       [ 1.02878530e-04,  1.22144257e-04, -4.82958200e-05,
         2.25263644e-04, -6.36662147e-05,  3.74582287e-05,
        -9.91911570e-05,  2.80821578e-05,  7.31551780e-05],
       [-4.72598459e-05,  9.03010873e-05, -1.44001827e-04,
        -6.36662147e-05,  2.29110098e-04,  3.68546155e-05,
        -3.48056740e-05,  7.68807205e-05,  3.17988063e-05],
       [-1.32023599e-04,  3.51360778e-05,  1.00355855e-04,
         3

In [47]:
res_2

      fun: 1010.7210601094179
 hess_inv: array([[ 0.02218107, -0.05384471, -0.00510658,  0.03010834, -0.02788944,
         0.02630274,  0.07033734,  0.01175199,  0.02527853],
       [-0.05384471,  0.13374105,  0.01279464, -0.0753933 ,  0.07101132,
        -0.0657801 , -0.17585405, -0.02910306, -0.06522286],
       [-0.00510658,  0.01279464,  0.00183102, -0.00679731,  0.00618626,
        -0.00666073, -0.0162846 , -0.0026312 , -0.0063262 ],
       [ 0.03010834, -0.0753933 , -0.00679731,  0.04310424, -0.04116899,
         0.03703269,  0.10004078,  0.01665646,  0.0374181 ],
       [-0.02788944,  0.07101132,  0.00618626, -0.04116899,  0.04003879,
        -0.03501402, -0.09517482, -0.01588423, -0.03635013],
       [ 0.02630274, -0.0657801 , -0.00666073,  0.03703269, -0.03501402,
         0.03274745,  0.0865023 ,  0.01432561,  0.03259572],
       [ 0.07033734, -0.17585405, -0.0162846 ,  0.10004078, -0.09517482,
         0.0865023 ,  0.23267264,  0.0386162 ,  0.08696181],
       [ 0.01175199, 

In [48]:
res_3

      fun: 1924.8381386234362
 hess_inv: array([[ 2.70862211e-03, -2.96335126e-03, -1.92061192e-03,
         8.65124036e-04,  2.07778445e-03,  7.52790624e-04,
        -6.08464763e-04, -6.33168727e-04,  7.70147433e-04],
       [-2.96335126e-03,  4.75455095e-03,  3.17684597e-03,
        -1.20939756e-03, -2.19614063e-03, -1.66540300e-03,
         7.15536101e-04, -8.94569206e-04, -8.76518330e-04],
       [-1.92061192e-03,  3.17684597e-03,  2.79402968e-03,
        -1.38735993e-03, -1.05933209e-03, -1.25687051e-03,
         9.51163210e-04, -1.17069994e-03, -4.50982626e-04],
       [ 8.65124036e-04, -1.20939756e-03, -1.38735993e-03,
         1.00005631e-03,  4.51174155e-04,  3.64198878e-04,
        -4.13776859e-04,  5.00539410e-04,  1.17194709e-04],
       [ 2.07778445e-03, -2.19614063e-03, -1.05933209e-03,
         4.51174155e-04,  2.31623850e-03,  5.86340944e-04,
        -2.57442030e-05, -1.24649073e-03,  1.23021394e-04],
       [ 7.52790624e-04, -1.66540300e-03, -1.25687051e-03,
         3

In [49]:
res_4

      fun: 3792.7303370142813
 hess_inv: array([[ 3.96321489e-05, -7.46273431e-06,  3.37062120e-05,
        -4.71875096e-05,  1.22402185e-05,  5.78237983e-05,
        -6.06594191e-05,  3.60873722e-05,  1.53287264e-06],
       [-7.46273431e-06,  2.01090963e-05, -1.64544748e-05,
         2.78009861e-05, -2.70667004e-05, -4.31583003e-05,
         2.73266721e-05, -1.79960640e-05,  1.45755491e-05],
       [ 3.37062120e-05, -1.64544748e-05,  4.54727855e-05,
        -6.30516762e-05,  2.89870215e-05,  8.57770975e-05,
        -7.72660994e-05,  4.65436035e-05, -1.23028431e-05],
       [-4.71875096e-05,  2.78009861e-05, -6.30516762e-05,
         1.06098772e-04, -5.79359140e-05, -1.60345559e-04,
         1.14835474e-04, -7.62615364e-05,  3.15056406e-05],
       [ 1.22402185e-05, -2.70667004e-05,  2.89870215e-05,
        -5.79359140e-05,  5.67911903e-05,  9.22783068e-05,
        -4.88643635e-05,  3.40257893e-05, -3.60403490e-05],
       [ 5.78237983e-05, -4.31583003e-05,  8.57770975e-05,
        -1

In [110]:
mean_3cluster = []
std_3cluster = []
mean_3cluster.append(mean_3_one)
mean_3cluster.append(mean_3_two)
mean_3cluster.append(mean_3_three)
std_3cluster.append(std_3_one)
std_3cluster.append(std_3_two)
std_3cluster.append(std_3_three)


In [111]:
theta_3cluster = []
theta_3cluster.append(df_3_one.shape[0] / data_3cluster.shape[0])
theta_3cluster.append(df_3_two.shape[0] / data_3cluster.shape[0])
theta_3cluster.append(df_3_three.shape[0] / data_3cluster.shape[0])

In [150]:
theta_3cluster

[0.8274742008640015, 0.1233326144213739, 0.04919318471462463]

In [112]:
beta_3cluster = []
beta_3cluster.append([-2.08215274,  0.42792156,  0.12208375,  0.11702509,  0.00999331,
        0.04675314, -0.06323514, -0.99569225,  0.1493836 ])
beta_3cluster.append([ 2.02049388,  0.58806615,  0.12565736,  0.11863238,  0.32397264,
        0.0330871 ,  0.2062005 , -1.05314573,  0.18988912])
beta_3cluster.append([ 1.83949155,  0.28127776,  0.10843109,  0.01705466,  0.16360347,
        0.01680751, -0.24775001, -0.36514211,  0.19182447])

In [51]:
mean_4cluster = []
std_4cluster = []
mean_4cluster.append(mean_4_one)
mean_4cluster.append(mean_4_two)
mean_4cluster.append(mean_4_three)
mean_4cluster.append(mean_4_four)
std_4cluster.append(std_4_one)
std_4cluster.append(std_4_two)
std_4cluster.append(std_4_three)
std_4cluster.append(std_4_four)

In [52]:
theta_4cluster = []
theta_4cluster.append(df_4_one.shape[0] / data_4cluster.shape[0])
theta_4cluster.append(df_4_two.shape[0] / data_4cluster.shape[0])
theta_4cluster.append(df_4_three.shape[0] / data_4cluster.shape[0])
theta_4cluster.append(df_4_four.shape[0] / data_4cluster.shape[0])

In [53]:
beta_4cluster = []
beta_4cluster.append([-1.59550736,  0.46089075,  0.11816455,  0.11046467,  0.0717849 ,
        0.03798292, -0.05805397, -1.03614344,  0.17266994])
beta_4cluster.append([-1.94242923,  0.17459783,  0.06103661,  0.01711175, -0.14394501,
        0.02369744, -0.14903244, -0.01516122,  0.14749104])
beta_4cluster.append([-2.20624549e+00,  3.38743861e-01,  1.13860140e-01,  1.37717589e-01,
       -3.75356203e-03,  3.76205415e-02, -8.96334726e-02, -4.35882283e+00,
        9.48561237e-02])
beta_4cluster.append([-2.44854238,  0.46847782,  0.13352973,  0.07686038, -0.03170865,
        0.06867889, -0.07128673, -4.84973152,  0.13216368])

In [169]:
mean_spec_cluster = []
std_spec_cluster = []
mean_spec_cluster.append(mean_spec_one)
mean_spec_cluster.append(mean_spec_two)
mean_spec_cluster.append(mean_spec_three)
mean_spec_cluster.append(mean_spec_four)
std_spec_cluster.append(std_spec_one)
std_spec_cluster.append(std_spec_two)
std_spec_cluster.append(std_spec_three)
std_spec_cluster.append(std_spec_four)

In [181]:
beta_spec = []
beta_spec.append([-1.59550736,  0.46089075,  0.11816455,  0.11046467,  0.0717849 ,
        0.03798292, -0.05805397, -1.03614344,  0.17266994])
beta_spec.append([-1.94242923,  0.17459783,  0.06103661,  0.01711175, -0.14394501,
        0.02369744, -0.14903244, -0.01516122,  0.14749104])
beta_spec.append([-2.32809247e+01,  1.91737735e-01,  4.06044319e-02,  3.73399985e-02,
       -2.99671576e-01,  4.59778762e-03, -2.56169928e-01, -7.50579092e+01,
        5.86493290e-02])
beta_spec.append([-2.44854238,  0.46847782,  0.13352973,  0.07686038, -0.03170865,
        0.06867889, -0.07128673, -4.84973152,  0.13216368])

In [231]:
beta_spec

[[-1.59550736,
  0.46089075,
  0.11816455,
  0.11046467,
  0.0717849,
  0.03798292,
  -0.05805397,
  -1.03614344,
  0.17266994],
 [-1.94242923,
  0.17459783,
  0.06103661,
  0.01711175,
  -0.14394501,
  0.02369744,
  -0.14903244,
  -0.01516122,
  0.14749104],
 [-23.2809247,
  0.191737735,
  0.0406044319,
  0.0373399985,
  -0.299671576,
  0.00459778762,
  -0.256169928,
  -75.0579092,
  0.058649329],
 [-2.44854238,
  0.46847782,
  0.13352973,
  0.07686038,
  -0.03170865,
  0.06867889,
  -0.07128673,
  -4.84973152,
  0.13216368]]

In [230]:
theta_spec_cluster

[0.12256141795580652,
 0.018926991222738533,
 0.040860341548536364,
 0.8176512492729185]

# Spec cluster

# Assortment for unknown type

In [182]:
df = pd.read_csv('data1.csv')
assortment1 = get_Assortment(df, mean_spec_cluster, std_spec_cluster, beta_spec, theta_spec_cluster)

Problem solved in 99489.000000 milliseconds
The objective value is  15.53181842459889


In [183]:
df = pd.read_csv('data2.csv')
assortment2 = get_Assortment(df, mean_spec_cluster, std_spec_cluster, beta_spec, theta_spec_cluster)

Problem solved in 311111.000000 milliseconds
The objective value is  18.937502974555137


In [184]:
df = pd.read_csv('data3.csv')
assortment3 = get_Assortment(df, mean_spec_cluster, std_spec_cluster, beta_spec, theta_spec_cluster)

Problem solved in 70475.000000 milliseconds
The objective value is  17.69125400447728


In [185]:
df = pd.read_csv('data4.csv')
assortment4 = get_Assortment(df, mean_spec_cluster, std_spec_cluster, beta_spec, theta_spec_cluster)

Problem solved in 1195044.000000 milliseconds
The objective value is  14.136094907811291


In [186]:
def get_unknown_type_Rev(assortment,df,mean,std,beta):
    # need to use mean riginal and std original here
    # use original beta here
    for i in range(8):
        df.iloc[:,i] = (df.iloc[:,i] - mean[i]) / std[i]
    for i in range(len(assortment)):
        if assortment[i] == 0:
        #     temp_df = pd.DataFrame(data2_original.iloc[i], columns=['prop_starrating', 'prop_review_score', 'prop_brand_bool',
        #    'prop_location_score', 'prop_accesibility_score',
        #    'prop_log_historical_price', 'price_usd', 'promotion_flag'])
        #     # data2_assotment = pd.concat([data2_assortment, temp_df])
            # data2_assortment = data2_assortment.append(temp_df)
            df = df.drop([i])
    features = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score', 'prop_accesibility_score',
           'prop_log_historical_price', 'price_usd', 'promotion_flag']
    original_mean = {}
    original_std = {}
    for i in range(len(features)):
        original_mean[features[i]] = mean[i]
        original_std[features[i]] = std[i]
    def expected_revenue_original(df, beta):
        num_hotel = df.shape[0]
        base = 1
        top = 0
        for i in range(num_hotel):
            u = 0
            u += beta[0]
            for j in range(1,9):
                u += beta[j] * df.iloc[i][j-1]
            # print(u)
            v = np.exp(u) 
            base += v
            p = df.iloc[i][-2] * original_std['price_usd'] + original_mean['price_usd']
            top += v*p
        expected_revenue = top / base 
        print('expected revenue is', expected_revenue)
    expected_revenue_original(df, beta)

In [187]:
# assortment for data1
beta = [-1.74662743,  0.40812391,  0.10876074,  0.10138235,  0.02202491,
        0.04344406, -0.06686938, -1.33110656,  0.15977641]

# Data 1 for unknown type

In [188]:
df = pd.read_csv('data1.csv')
get_unknown_type_Rev(assortment1,df,mean_original,std_original,beta)

expected revenue is 107.3396340843958


# Data 2 for unknown type

In [189]:
df = pd.read_csv('data2.csv')
get_unknown_type_Rev(assortment2,df,mean_original,std_original,beta)

expected revenue is 130.5562106874302


# Data 3 for unknown type

In [190]:
df = pd.read_csv('data3.csv')
get_unknown_type_Rev(assortment3,df,mean_original,std_original,beta)

expected revenue is 120.9612284936689


# Data 4 for unknown type

In [191]:
df = pd.read_csv('data4.csv')
get_unknown_type_Rev(assortment4,df,mean_original,std_original,beta)

expected revenue is 97.4088309422418


In [192]:
def revenue(assortment, beta):

    num_hotel = assortment.shape[0]

    base = 1
    top = 0

    for i in range(num_hotel):

        u = 0
        u += beta[0]

        for j in range(1,9):
            u += beta[j] * assortment.iloc[i][j-1]

        # print(u)
            
        v = np.exp(u) 
        base += v

        p = assortment.iloc[i][-2]
        top += v*p

    expected_revenue = top / base 

    return expected_revenue
def find_optimal(df,mean, std, beta):

    for feature in new_features:
        df[feature] = (df[feature] - mean[feature]) / std[feature]
    # for i in range(len(new_features)):
    #    df[new_features[i]] = (df[new_features[i]] - mean[i]) / std[i]
    df = df.sort_values(["price_usd"],ascending=False)
    sorted_df = df.reset_index(drop=True)
    
    num = sorted_df.shape[0]
    revenue_list = []

    for i in range(num):
        assortment = sorted_df.head(i+1)
        revenue_list.append(revenue(assortment,beta))


    maximum = max(revenue_list)
    for i in range(len(revenue_list)):
        if revenue_list[i] == maximum:
            print('The optimal assortment is a display of ', (i+1), ' highest priced products.') 
def expected_revenue(num, mean, std, df, beta):

    optimal_assortment = df.head(num)

    num_hotel = optimal_assortment.shape[0]

    base = 1
    top = 0

    for i in range(num_hotel):

        u = 0
        u += beta[0]

        for j in range(1,9):
            u += beta[j] * optimal_assortment.iloc[i][j-1]

        # print(u)
            
        v = np.exp(u) 
        base += v

        p = optimal_assortment.iloc[i][-2] * std['price_usd'] + mean['price_usd']
        top += v*p

    expected_revenue = top / base 


    print('expected revenue is', expected_revenue)

features = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score', 'prop_accesibility_score',
       'prop_log_historical_price', 'price_usd', 'promotion_flag']

one_mean = {}
one_std = {}
two_mean = {}
two_std = {}
three_mean = {}
three_std = {}
four_mean = {}
four_std = {}

for i in range(len(features)):
       
    one_mean[features[i]] = mean_spec_one[i]
    one_std[features[i]] = std_spec_one[i]
    two_mean[features[i]] = mean_spec_two[i]
    two_std[features[i]] = std_spec_two[i]
    three_mean[features[i]] = mean_spec_three[i]
    three_std[features[i]] = std_spec_three[i]
    four_mean[features[i]] = mean_spec_four[i]
    four_std[features[i]] = std_spec_four[i]

new_features = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score', 'prop_accesibility_score',
       'prop_log_historical_price', 'price_usd', 'promotion_flag']

# Data 1 when knowing the type

In [193]:
data1 = pd.read_csv('data1.csv')
find_optimal(data1, one_mean, one_std, beta_spec[0])

The optimal assortment is a display of  3  highest priced products.


In [194]:
expected_revenue(3, one_mean, one_std, data1, beta_spec[0])

expected revenue is 50.22221267036282


In [195]:
data2 = pd.read_csv('data1.csv')
find_optimal(data2, two_mean, two_std, beta_spec[1])

The optimal assortment is a display of  1  highest priced products.


In [196]:
expected_revenue(1, two_mean, two_std, data2, beta_spec[1])

expected revenue is 21.190809084136927


In [200]:
data3 = pd.read_csv('data1.csv')
find_optimal(data3, three_mean, three_std, beta_spec[2])

The optimal assortment is a display of  1  highest priced products.


In [201]:
expected_revenue(1, three_mean, three_std, data3, beta_spec[2])

expected revenue is 3.0481028030790428e-06


In [202]:
data4 = pd.read_csv('data1.csv')
find_optimal(data4, four_mean, four_std, beta_spec[3])

The optimal assortment is a display of  8  highest priced products.


In [203]:
expected_revenue(8, four_mean, four_std, data4, beta_spec[3])

expected revenue is 60.73677000846684


# Data 2 when knowing the type

In [204]:
data1 = pd.read_csv('data2.csv')
find_optimal(data1, one_mean, one_std, beta_spec[0])

The optimal assortment is a display of  3  highest priced products.


In [205]:
expected_revenue(3, one_mean, one_std, data1, beta_spec[0])

expected revenue is 95.2280514959225


In [206]:
data2 = pd.read_csv('data2.csv')
find_optimal(data2, two_mean, two_std, beta_spec[1])

The optimal assortment is a display of  3  highest priced products.


In [207]:
expected_revenue(3, two_mean, two_std, data2, beta_spec[1])

expected revenue is 72.2625063412781


In [208]:
data3 = pd.read_csv('data2.csv')
find_optimal(data3, three_mean, three_std, beta_spec[2])

The optimal assortment is a display of  3  highest priced products.


In [209]:
expected_revenue(3, three_mean, three_std, data3, beta_spec[2])

expected revenue is 4.717336156204596e-05


In [210]:
data4 = pd.read_csv('data2.csv')
find_optimal(data4, four_mean, four_std, beta_spec[3])

The optimal assortment is a display of  7  highest priced products.


In [211]:
expected_revenue(7, four_mean, four_std, data4, beta_spec[3])

expected revenue is 80.9750431152258


# Data 3 when knowing the type

In [212]:
data1 = pd.read_csv('data3.csv')
find_optimal(data1, one_mean, one_std, beta_spec[0])

The optimal assortment is a display of  8  highest priced products.


In [213]:
expected_revenue(8, one_mean, one_std, data1, beta_spec[0])

expected revenue is 102.53211015470683


In [214]:
data2 = pd.read_csv('data3.csv')
find_optimal(data2, two_mean, two_std, beta_spec[1])

The optimal assortment is a display of  2  highest priced products.


In [215]:
expected_revenue(2, two_mean, two_std, data2, beta_spec[1])

expected revenue is 39.73228698385515


In [216]:
data3 = pd.read_csv('data3.csv')
find_optimal(data3, three_mean, three_std, beta_spec[2])

The optimal assortment is a display of  3  highest priced products.


In [217]:
expected_revenue(3, three_mean, three_std, data3, beta_spec[2])

expected revenue is 8.2158780216051e-06


In [220]:
data4 = pd.read_csv('data3.csv')
find_optimal(data4, four_mean, four_std, beta_spec[3])

The optimal assortment is a display of  12  highest priced products.


In [221]:
expected_revenue(12, four_mean, four_std, data4, beta_spec[3])

expected revenue is 63.03778328790421


# Data 4 when knowing the type

In [222]:
data1 = pd.read_csv('data4.csv')
find_optimal(data1, one_mean, one_std, beta_spec[0])

The optimal assortment is a display of  1  highest priced products.


In [223]:
expected_revenue(1, one_mean, one_std, data1, beta_spec[0])

expected revenue is 36.688574026859406


In [224]:
data2 = pd.read_csv('data4.csv')
find_optimal(data2, two_mean, two_std, beta_spec[1])

The optimal assortment is a display of  1  highest priced products.


In [225]:
expected_revenue(1, two_mean, two_std, data2, beta_spec[1])

expected revenue is 19.756013117429823


In [226]:
data3 = pd.read_csv('data4.csv')
find_optimal(data3, three_mean, three_std, beta_spec[2])

The optimal assortment is a display of  1  highest priced products.


In [227]:
expected_revenue(1, three_mean, three_std, data3, beta_spec[2])

expected revenue is 0.009354698538806955


In [228]:
data4 = pd.read_csv('data4.csv')
find_optimal(data4, four_mean, four_std, beta_spec[3])

The optimal assortment is a display of  3  highest priced products.


In [229]:
expected_revenue(3, four_mean, four_std, data4, beta_spec[3])

expected revenue is 57.48420476680186


# 3 cluster

# Assortment for unknown type 

In [113]:
df = pd.read_csv('data1.csv')
assortment1 = get_Assortment(df, mean_3cluster, std_3cluster, beta_3cluster, theta_3cluster)
assortment1

Problem solved in 439574.000000 milliseconds
The objective value is  98.98910373709943


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0]

In [114]:
df = pd.read_csv('data2.csv')
assortment2 = get_Assortment(df, mean_3cluster, std_3cluster, beta_3cluster, theta_3cluster)
assortment2

Problem solved in 1460046.000000 milliseconds
The objective value is  118.6764664612857


[1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0]

In [115]:
df = pd.read_csv('data3.csv')
assortment3 = get_Assortment(df, mean_3cluster, std_3cluster, beta_3cluster, theta_3cluster)
assortment3

Problem solved in 173568.000000 milliseconds
The objective value is  108.94736733644243


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0]

In [116]:
df = pd.read_csv('data4.csv')
assortment4 = get_Assortment(df, mean_3cluster, std_3cluster, beta_3cluster, theta_3cluster)
assortment4

Problem solved in 216319.000000 milliseconds
The objective value is  88.8759555918848


[0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0]

In [117]:
def get_unknown_type_Rev(assortment,df,mean,std,beta):
    # need to use mean riginal and std original here
    # use original beta here
    for i in range(8):
        df.iloc[:,i] = (df.iloc[:,i] - mean[i]) / std[i]
    for i in range(len(assortment)):
        if assortment[i] == 0:
        #     temp_df = pd.DataFrame(data2_original.iloc[i], columns=['prop_starrating', 'prop_review_score', 'prop_brand_bool',
        #    'prop_location_score', 'prop_accesibility_score',
        #    'prop_log_historical_price', 'price_usd', 'promotion_flag'])
        #     # data2_assotment = pd.concat([data2_assortment, temp_df])
            # data2_assortment = data2_assortment.append(temp_df)
            df = df.drop([i])
    features = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score', 'prop_accesibility_score',
           'prop_log_historical_price', 'price_usd', 'promotion_flag']
    original_mean = {}
    original_std = {}
    for i in range(len(features)):
        original_mean[features[i]] = mean[i]
        original_std[features[i]] = std[i]
    def expected_revenue_original(df, beta):
        num_hotel = df.shape[0]
        base = 1
        top = 0
        for i in range(num_hotel):
            u = 0
            u += beta[0]
            for j in range(1,9):
                u += beta[j] * df.iloc[i][j-1]
            # print(u)
            v = np.exp(u) 
            base += v
            p = df.iloc[i][-2] * original_std['price_usd'] + original_mean['price_usd']
            top += v*p
        expected_revenue = top / base 
        print('expected revenue is', expected_revenue)
    expected_revenue_original(df, beta)

In [118]:
beta = [-1.74662743,  0.40812391,  0.10876074,  0.10138235,  0.02202491,
        0.04344406, -0.06686938, -1.33110656,  0.15977641]

# Data 1 for unknown type

In [119]:
df = pd.read_csv('data1.csv')
get_unknown_type_Rev(assortment1,df,mean_original,std_original,beta)

expected revenue is 107.29109444913811


# Data 2 for unknown type

In [120]:
df = pd.read_csv('data2.csv')
get_unknown_type_Rev(assortment2,df,mean_original,std_original,beta)

expected revenue is 131.11344212920363


# Data 3 for unknown type

In [121]:
df = pd.read_csv('data3.csv')
get_unknown_type_Rev(assortment3,df,mean_original,std_original,beta)

expected revenue is 121.01835939563993


# Data 4 for unknown type

In [122]:
df = pd.read_csv('data4.csv')
get_unknown_type_Rev(assortment4,df,mean_original,std_original,beta)

expected revenue is 97.4088309422418


In [123]:
def revenue(assortment, beta):

    num_hotel = assortment.shape[0]

    base = 1
    top = 0

    for i in range(num_hotel):

        u = 0
        u += beta[0]

        for j in range(1,9):
            u += beta[j] * assortment.iloc[i][j-1]

        # print(u)
            
        v = np.exp(u) 
        base += v

        p = assortment.iloc[i][-2]
        top += v*p

    expected_revenue = top / base 

    return expected_revenue
def find_optimal(df,mean, std, beta):

    for feature in new_features:
        df[feature] = (df[feature] - mean[feature]) / std[feature]
    # for i in range(len(new_features)):
    #    df[new_features[i]] = (df[new_features[i]] - mean[i]) / std[i]
    df = df.sort_values(["price_usd"],ascending=False)
    sorted_df = df.reset_index(drop=True)
    
    num = sorted_df.shape[0]
    revenue_list = []

    for i in range(num):
        assortment = sorted_df.head(i+1)
        revenue_list.append(revenue(assortment,beta))


    maximum = max(revenue_list)
    for i in range(len(revenue_list)):
        if revenue_list[i] == maximum:
            print('The optimal assortment is a display of ', (i+1), ' highest priced products.') 
def expected_revenue(num, mean, std, df, beta):

    optimal_assortment = df.head(num)

    num_hotel = optimal_assortment.shape[0]

    base = 1
    top = 0

    for i in range(num_hotel):

        u = 0
        u += beta[0]

        for j in range(1,9):
            u += beta[j] * optimal_assortment.iloc[i][j-1]

        # print(u)
            
        v = np.exp(u) 
        base += v

        p = optimal_assortment.iloc[i][-2] * std['price_usd'] + mean['price_usd']
        top += v*p

    expected_revenue = top / base 


    print('expected revenue is', expected_revenue)

features = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score', 'prop_accesibility_score',
       'prop_log_historical_price', 'price_usd', 'promotion_flag']

one_mean = {}
one_std = {}
two_mean = {}
two_std = {}
three_mean = {}
three_std = {}


for i in range(len(features)):
       
    one_mean[features[i]] = mean_3_one[i]
    one_std[features[i]] = std_3_one[i]
    two_mean[features[i]] = mean_3_two[i]
    two_std[features[i]] = std_3_two[i]
    three_mean[features[i]] = mean_3_three[i]
    three_std[features[i]] = std_3_three[i]


new_features = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score', 'prop_accesibility_score',
       'prop_log_historical_price', 'price_usd', 'promotion_flag']

# Data 1 when knowing the type

In [124]:
data1 = pd.read_csv('data1.csv')
find_optimal(data1, one_mean, one_std, beta_3cluster[0])

The optimal assortment is a display of  7  highest priced products.


In [125]:
expected_revenue(7, one_mean, one_std, data1, beta_3cluster[0])

expected revenue is 67.21104633429165


In [126]:
data2 = pd.read_csv('data1.csv')
find_optimal(data2, two_mean, two_std, beta_3cluster[1])

The optimal assortment is a display of  1  highest priced products.


In [127]:
expected_revenue(1, two_mean, two_std, data2, beta_3cluster[1])

expected revenue is 116.46530195541274


In [128]:
data3 = pd.read_csv('data1.csv')
find_optimal(data3, three_mean, three_std, beta_3cluster[2])

The optimal assortment is a display of  1  highest priced products.


In [129]:
expected_revenue(1, three_mean, three_std, data3, beta_3cluster[2])

expected revenue is 118.61326728080468


# Data 2 when knowing the type

In [130]:
data1 = pd.read_csv('data2.csv')
find_optimal(data1, one_mean, one_std, beta_3cluster[0])

The optimal assortment is a display of  3  highest priced products.


In [131]:
expected_revenue(3, one_mean, one_std, data1, beta_3cluster[0])

expected revenue is 71.30801404477035


In [132]:
data2 = pd.read_csv('data2.csv')
find_optimal(data2, two_mean, two_std, beta_3cluster[1])

The optimal assortment is a display of  1  highest priced products.


In [134]:
expected_revenue(1, two_mean, two_std, data2, beta_3cluster[1])

expected revenue is 215.10739040432304


In [135]:
data3 = pd.read_csv('data2.csv')
find_optimal(data3, three_mean, three_std, beta_3cluster[2])

The optimal assortment is a display of  1  highest priced products.


In [136]:
expected_revenue(1, three_mean, three_std, data3, beta_3cluster[2])

expected revenue is 225.19728779793326


# Data 3 when knowing the type

In [137]:
data1 = pd.read_csv('data3.csv')
find_optimal(data1, one_mean, one_std, beta_3cluster[0])

The optimal assortment is a display of  9  highest priced products.


In [138]:
expected_revenue(9, one_mean, one_std, data1, beta_3cluster[0])

expected revenue is 84.39048093239713


In [139]:
data2 = pd.read_csv('data3.csv')
find_optimal(data2, two_mean, two_std, beta_3cluster[1])

The optimal assortment is a display of  2  highest priced products.


In [140]:
expected_revenue(2, two_mean, two_std, data2, beta_3cluster[1])

expected revenue is 146.67162852755308


In [141]:
data3 = pd.read_csv('data3.csv')
find_optimal(data3, three_mean, three_std, beta_3cluster[2])

The optimal assortment is a display of  1  highest priced products.


In [142]:
expected_revenue(1, three_mean, three_std, data3, beta_3cluster[2])

expected revenue is 119.96921028047899


# Data 4 when knowing the type

In [143]:
data1 = pd.read_csv('data4.csv')
find_optimal(data1, one_mean, one_std, beta_3cluster[0])

The optimal assortment is a display of  2  highest priced products.


In [144]:
expected_revenue(2, one_mean, one_std, data1, beta_3cluster[0])

expected revenue is 27.69700493238335


In [145]:
data2 = pd.read_csv('data4.csv')
find_optimal(data2, two_mean, two_std, beta_3cluster[1])

The optimal assortment is a display of  1  highest priced products.


In [146]:
expected_revenue(1, two_mean, two_std, data2, beta_3cluster[1])

expected revenue is 68.44321534637793


In [147]:
data3 = pd.read_csv('data4.csv')
find_optimal(data3, three_mean, three_std, beta_3cluster[2])

The optimal assortment is a display of  1  highest priced products.


In [148]:
expected_revenue(1, three_mean, three_std, data3, beta_3cluster[2])

expected revenue is 68.72451466090337


# 4 cluster

# Assortment for unknown type

In [106]:
df = pd.read_csv('data1.csv')
assortment1 = get_Assortment(df, mean_4cluster, std_4cluster, beta_4cluster, theta_4cluster)
assortment1

Problem solved in 1470.000000 milliseconds
The objective value is  78.11959734687359


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0]

In [56]:
df = pd.read_csv('data2.csv')
assortment2 = get_Assortment(df, mean_4cluster, std_4cluster, beta_4cluster, theta_4cluster)
assortment2

Problem solved in 6511.000000 milliseconds
The objective value is  96.11959553327092


[1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [57]:
df = pd.read_csv('data3.csv')
assortment3 = get_Assortment(df, mean_4cluster, std_4cluster, beta_4cluster, theta_4cluster)
assortment3

Problem solved in 7720.000000 milliseconds
The objective value is  88.12637141520936


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0]

In [59]:
df = pd.read_csv('data4.csv')
assortment4 = get_Assortment(df, mean_4cluster, std_4cluster, beta_4cluster, theta_4cluster)
assortment4

Problem solved in 10533.000000 milliseconds
The objective value is  70.85190786353445


[0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0]

In [60]:
def get_unknown_type_Rev(assortment,df,mean,std,beta):
    # need to use mean riginal and std original here
    # use original beta here
    for i in range(8):
        df.iloc[:,i] = (df.iloc[:,i] - mean[i]) / std[i]
    for i in range(len(assortment)):
        if assortment[i] == 0:
        #     temp_df = pd.DataFrame(data2_original.iloc[i], columns=['prop_starrating', 'prop_review_score', 'prop_brand_bool',
        #    'prop_location_score', 'prop_accesibility_score',
        #    'prop_log_historical_price', 'price_usd', 'promotion_flag'])
        #     # data2_assotment = pd.concat([data2_assortment, temp_df])
            # data2_assortment = data2_assortment.append(temp_df)
            df = df.drop([i])
    features = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score', 'prop_accesibility_score',
           'prop_log_historical_price', 'price_usd', 'promotion_flag']
    original_mean = {}
    original_std = {}
    for i in range(len(features)):
        original_mean[features[i]] = mean[i]
        original_std[features[i]] = std[i]
    def expected_revenue_original(df, beta):
        num_hotel = df.shape[0]
        base = 1
        top = 0
        for i in range(num_hotel):
            u = 0
            u += beta[0]
            for j in range(1,9):
                u += beta[j] * df.iloc[i][j-1]
            # print(u)
            v = np.exp(u) 
            base += v
            p = df.iloc[i][-2] * original_std['price_usd'] + original_mean['price_usd']
            top += v*p
        expected_revenue = top / base 
        print('expected revenue is', expected_revenue)
    expected_revenue_original(df, beta)

In [61]:
# assortment for data1
beta = [-1.74662743,  0.40812391,  0.10876074,  0.10138235,  0.02202491,
        0.04344406, -0.06686938, -1.33110656,  0.15977641]

# Data 1 for unknown type

In [108]:
df = pd.read_csv('data1.csv')
get_unknown_type_Rev(assortment1,df,mean_original,std_original,beta)

expected revenue is 107.3396340843958


# Data 2 for unknown type

In [63]:
df = pd.read_csv('data2.csv')
get_unknown_type_Rev(assortment2,df,mean_original,std_original,beta)

expected revenue is 130.5562106874302


# Data 3 for unknown type

In [64]:
df = pd.read_csv('data3.csv')
get_unknown_type_Rev(assortment3,df,mean_original,std_original,beta)

expected revenue is 120.9612284936689


# Data 4 for unknown type

In [65]:
df = pd.read_csv('data4.csv')
get_unknown_type_Rev(assortment4,df,mean_original,std_original,beta)

expected revenue is 97.4088309422418


In [66]:
def revenue(assortment, beta):

    num_hotel = assortment.shape[0]

    base = 1
    top = 0

    for i in range(num_hotel):

        u = 0
        u += beta[0]

        for j in range(1,9):
            u += beta[j] * assortment.iloc[i][j-1]

        # print(u)
            
        v = np.exp(u) 
        base += v

        p = assortment.iloc[i][-2]
        top += v*p

    expected_revenue = top / base 

    return expected_revenue
def find_optimal(df,mean, std, beta):

    for feature in new_features:
        df[feature] = (df[feature] - mean[feature]) / std[feature]
    # for i in range(len(new_features)):
    #    df[new_features[i]] = (df[new_features[i]] - mean[i]) / std[i]
    df = df.sort_values(["price_usd"],ascending=False)
    sorted_df = df.reset_index(drop=True)
    
    num = sorted_df.shape[0]
    revenue_list = []

    for i in range(num):
        assortment = sorted_df.head(i+1)
        revenue_list.append(revenue(assortment,beta))


    maximum = max(revenue_list)
    for i in range(len(revenue_list)):
        if revenue_list[i] == maximum:
            print('The optimal assortment is a display of ', (i+1), ' highest priced products.') 
def expected_revenue(num, mean, std, df, beta):

    optimal_assortment = df.head(num)

    num_hotel = optimal_assortment.shape[0]

    base = 1
    top = 0

    for i in range(num_hotel):

        u = 0
        u += beta[0]

        for j in range(1,9):
            u += beta[j] * optimal_assortment.iloc[i][j-1]

        # print(u)
            
        v = np.exp(u) 
        base += v

        p = optimal_assortment.iloc[i][-2] * std['price_usd'] + mean['price_usd']
        top += v*p

    expected_revenue = top / base 


    print('expected revenue is', expected_revenue)

features = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score', 'prop_accesibility_score',
       'prop_log_historical_price', 'price_usd', 'promotion_flag']

one_mean = {}
one_std = {}
two_mean = {}
two_std = {}
three_mean = {}
three_std = {}
four_mean = {}
four_std = {}

for i in range(len(features)):
       
    one_mean[features[i]] = mean_4_one[i]
    one_std[features[i]] = std_4_one[i]
    two_mean[features[i]] = mean_4_two[i]
    two_std[features[i]] = std_4_two[i]
    three_mean[features[i]] = mean_4_three[i]
    three_std[features[i]] = std_4_three[i]
    four_mean[features[i]] = mean_4_four[i]
    four_std[features[i]] = std_4_four[i]

new_features = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score', 'prop_accesibility_score',
       'prop_log_historical_price', 'price_usd', 'promotion_flag']

# Data 1 when knowing the type

In [69]:
data1 = pd.read_csv('data1.csv')
find_optimal(data1, one_mean, one_std, beta_4cluster[0])

The optimal assortment is a display of  6  highest priced products.


In [70]:
expected_revenue(6, one_mean, one_std, data1, beta_4cluster[0])

expected revenue is 76.09948094662586


In [71]:
data2 = pd.read_csv('data1.csv')
find_optimal(data2, two_mean, two_std, beta_4cluster[1])

The optimal assortment is a display of  4  highest priced products.


In [72]:
expected_revenue(4, two_mean, two_std, data2, beta_4cluster[1])

expected revenue is 56.399565981831635


In [73]:
data3 = pd.read_csv('data1.csv')
find_optimal(data3, three_mean, three_std, beta_4cluster[2])

The optimal assortment is a display of  1  highest priced products.


In [74]:
expected_revenue(1, three_mean, three_std, data3, beta_4cluster[2])

expected revenue is 17.825977219784924


In [75]:
data4 = pd.read_csv('data1.csv')
find_optimal(data4, four_mean, four_std, beta_4cluster[3])

The optimal assortment is a display of  1  highest priced products.


In [77]:
expected_revenue(1, four_mean, four_std, data4, beta_4cluster[3])

expected revenue is 14.40635043647111


# Data 2 when knowing the type

In [78]:
data1 = pd.read_csv('data2.csv')
find_optimal(data1, one_mean, one_std, beta_4cluster[0])

The optimal assortment is a display of  3  highest priced products.


In [79]:
expected_revenue(3, one_mean, one_std, data1, beta_4cluster[0])

expected revenue is 95.08681622346974


In [80]:
data2 = pd.read_csv('data2.csv')
find_optimal(data2, two_mean, two_std, beta_4cluster[1])

The optimal assortment is a display of  3  highest priced products.


In [81]:
expected_revenue(3, two_mean, two_std, data2, beta_4cluster[1])

expected revenue is 67.26774225862488


In [82]:
data3 = pd.read_csv('data2.csv')
find_optimal(data3, three_mean, three_std, beta_4cluster[2])

The optimal assortment is a display of  3  highest priced products.


In [83]:
expected_revenue(3, three_mean, three_std, data3, beta_4cluster[2])

expected revenue is 79.24615991027659


In [84]:
data4 = pd.read_csv('data2.csv')
find_optimal(data4, four_mean, four_std, beta_4cluster[3])

The optimal assortment is a display of  3  highest priced products.


In [85]:
expected_revenue(3, four_mean, four_std, data4, beta_4cluster[3])

expected revenue is 73.5583729290145


# Data 3 when knowing the type

In [86]:
data1 = pd.read_csv('data3.csv')
find_optimal(data1, one_mean, one_std, beta_4cluster[0])

The optimal assortment is a display of  9  highest priced products.


In [89]:
expected_revenue(9, one_mean, one_std, data1, beta_4cluster[0])

expected revenue is 103.04253226744395


In [90]:
data2 = pd.read_csv('data3.csv')
find_optimal(data2, two_mean, two_std, beta_4cluster[1])

The optimal assortment is a display of  3  highest priced products.


In [91]:
expected_revenue(3, two_mean, two_std, data2, beta_4cluster[1])

expected revenue is 50.08825158709839


In [92]:
data3 = pd.read_csv('data3.csv')
find_optimal(data3, three_mean, three_std, beta_4cluster[2])

The optimal assortment is a display of  3  highest priced products.


In [93]:
expected_revenue(3, three_mean, three_std, data3, beta_4cluster[2])

expected revenue is 52.0725866801742


In [94]:
data4 = pd.read_csv('data3.csv')
find_optimal(data4, four_mean, four_std, beta_4cluster[3])

The optimal assortment is a display of  4  highest priced products.


In [95]:
expected_revenue(4, four_mean, four_std, data4, beta_4cluster[3])

expected revenue is 55.262678799566295


# Data 4 when knowing the type

In [96]:
data1 = pd.read_csv('data4.csv')
find_optimal(data1, one_mean, one_std, beta_4cluster[0])

The optimal assortment is a display of  2  highest priced products.


In [97]:
expected_revenue(2, one_mean, one_std, data1, beta_4cluster[0])

expected revenue is 37.166846933375595


In [98]:
data2 = pd.read_csv('data4.csv')
find_optimal(data2, two_mean, two_std, beta_4cluster[1])

The optimal assortment is a display of  1  highest priced products.


In [99]:
expected_revenue(1, two_mean, two_std, data2, beta_4cluster[1])

expected revenue is 17.896870188359003


In [100]:
data3 = pd.read_csv('data4.csv')
find_optimal(data3, three_mean, three_std, beta_4cluster[2])

The optimal assortment is a display of  1  highest priced products.


In [101]:
expected_revenue(1, three_mean, three_std, data3, beta_4cluster[2])

expected revenue is 23.0126797928701


In [102]:
data4 = pd.read_csv('data4.csv')
find_optimal(data4, four_mean, four_std, beta_4cluster[3])

The optimal assortment is a display of  1  highest priced products.


In [103]:
expected_revenue(1, four_mean, four_std, data4, beta_4cluster[3])

expected revenue is 24.956915436056192
